In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

In [2]:
aisles = pd.read_csv('/instacart/aisles.csv')
departments = pd.read_csv('/instacart/departments.csv')
orders = pd.read_csv('/instacart/orders.csv')
products = pd.read_csv('/instacart/products.csv')

In [3]:
order_products_prior = pd.read_csv('/instacart/order_products__prior.csv')
orderproducts_train = pd.read_csv('/instacart/order_products__train.csv')

In [4]:
order_products_prior.sample(3)

,order_id,product_id,add_to_cart_order,reordered
24272049,2559792,21137,1,1
7156970,755689,30561,12,1
30958322,3265562,47616,9,1


In [5]:
order_products_prior.shape

(32434489, 4)

In [6]:
orders.head()

,order_id,user_id,eval_set,order_number,order_dow,order_hour_of_day,days_since_prior_order
0,2539329,1,prior,1,2,8,NaN
1,2398795,1,prior,2,3,7,15.0
2,473747,1,prior,3,3,12,21.0
3,2254736,1,prior,4,4,7,29.0
4,431534,1,prior,5,4,15,28.0


In [7]:
orders2=orders[orders['eval_set']=='prior']

In [8]:
prd = orders2.merge(order_products_prior, on='order_id', how='left')
prd.head()

,order_id,user_id,eval_set,order_number,order_dow,order_hour_of_day,days_since_prior_order,product_id,add_to_cart_order,reordered
0,2539329,1,prior,1,2,8,NaN,196,1,0
1,2539329,1,prior,1,2,8,NaN,14084,2,0
2,2539329,1,prior,1,2,8,NaN,12427,3,0
3,2539329,1,prior,1,2,8,NaN,26088,4,0
4,2539329,1,prior,1,2,8,NaN,26405,5,0


In [9]:
prd.shape

(32434489, 10)

In [10]:
#prd = prd[['order_id', 'user_id','product_id','reordered']]

In [11]:
uxp = prd.groupby(['user_id', 'product_id'])[['order_id']].count()
uxp.columns = ['total_orders']
uxp = uxp.reset_index()

In [12]:
uxp.head()

,user_id,product_id,total_orders
0,1,196,10
1,1,10258,9
2,1,10326,1
3,1,12427,10
4,1,13032,3


## Last 10 order ratio

How many times a customer bought a product on its last 10 orders ?

In [13]:
prd['order_number_back'] = prd.groupby('user_id')['order_number'].transform(max) - prd.order_number +1 
prd10 = prd[prd.order_number_back <= 10]

C:\Users\anish\AppData\Local\Temp\ipykernel_4912\3899398049.py:1: FutureWarning: The provided callable <built-in function max> is currently using SeriesGroupBy.max. In a future version of pandas, the provided callable will be used directly. To keep current behavior pass the string "max" instead.
  prd['order_number_back'] = prd.groupby('user_id')['order_number'].transform(max) - prd.order_number +1


In [14]:
last_10 = prd10.groupby(['user_id','product_id'])[['order_id']].count()
last_10.columns = ['times_last10']
last_10['times_last10_ratio'] = last_10.times_last10 / 10

In [15]:
last_10.sample(10)

,,times_last10,times_last10_ratio
user_id,product_id,,
124602,45394,2,0.2
47564,30958,2,0.2
93541,215,1,0.1
147073,4595,3,0.3
199971,42307,2,0.2
60179,32455,2,0.2
106065,49621,1,0.1
24234,48205,1,0.1
205468,36724,1,0.1


In [16]:
uxp = uxp.merge(last_10 , on=['user_id', 'product_id'], how='left')
del [last_10, prd10]

In [17]:
uxp.sample(10)

,user_id,product_id,total_orders,times_last10,times_last10_ratio
813943,12793,19345,1,1.0,0.1
3565523,55195,21925,2,2.0,0.2
4844786,74937,24382,1,1.0,0.1
13038005,201990,18288,1,NaN,NaN
1746275,27159,31511,1,1.0,0.1
4043567,62732,36675,1,NaN,NaN
3190209,49474,36944,1,1.0,0.1
561920,8856,3957,1,NaN,NaN
6066725,93688,44628,1,1.0,0.1
8256752,127666,27966,4,2.0,0.2


## Reorder Ratio of Products

Remove products with less than 30 purchases

In [18]:
reorder= order_products_prior.groupby('product_id').filter(lambda x:x.shape[0]>30)

In [19]:
reorder = reorder.groupby('product_id')[['reordered']].mean()
reorder.columns = ['reorder_ratio']
reorder.head()

,reorder_ratio
product_id,
1,0.613391
2,0.133333
3,0.732852
4,0.446809
8,0.503030


In [20]:
uxp = uxp.merge(reorder, on='product_id', how='left')

In [21]:
uxp.sample(10)

,user_id,product_id,total_orders,times_last10,times_last10_ratio,reorder_ratio
2156036,33553,30558,3,3.0,0.3,0.556199
695107,10960,45870,1,1.0,0.1,0.332903
12008441,185980,39293,3,1.0,0.1,0.705479
5600273,86491,13062,2,NaN,NaN,0.657076
8227084,127167,33401,2,NaN,NaN,0.549442
4374770,67765,37247,3,NaN,NaN,0.242424
5947835,91823,38383,1,1.0,0.1,0.625735
6805392,105028,34969,27,9.0,0.9,0.639821
590365,9321,12341,13,4.0,0.4,0.744011
1790286,27854,27761,2,NaN,NaN,0.576481


In [22]:
uxp.describe()

,user_id,product_id,total_orders,times_last10,times_last10_ratio,reorder_ratio
count,1.330795e+07,1.330795e+07,1.330795e+07,9.223671e+06,9.223671e+06,1.313176e+07
mean,1.029987e+05,2.551351e+04,2.437226e+00,1.744288e+00,1.744288e-01,5.386909e-01
std,5.943677e+04,1.422429e+04,3.554528e+00,1.425065e+00,1.425065e-01,1.582521e-01
min,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e-01,0.000000e+00
25%,5.157900e+04,1.329200e+04,1.000000e+00,1.000000e+00,1.000000e-01,4.470310e-01
50%,1.027130e+05,2.564000e+04,1.000000e+00,1.000000e+00,1.000000e-01,5.618474e-01
75%,1.544500e+05,3.815700e+04,2.000000e+00,2.000000e+00,2.000000e-01,6.525929e-01
max,2.062090e+05,4.968800e+04,9.900000e+01,1.000000e+01,1.000000e+00,9.411765e-01


## Merged Ratio
### Combining the number of times a product repeated in a users order with the overall reorder ratio

In [23]:
uxp['Merged_Ratio']=uxp['times_last10']*uxp['reorder_ratio']

In [179]:
uxp.tail()

,user_id,product_id,total_orders,times_last10,times_last10_ratio,reorder_ratio,Merged_Ratio
13307948,206209,43961,3,3.0,0.3,0.630583,1.891748
13307949,206209,44325,1,1.0,0.1,0.401148,0.401148
13307950,206209,48370,1,1.0,0.1,0.699288,0.699288
13307951,206209,48697,1,1.0,0.1,0.357661,0.357661
13307952,206209,48742,2,2.0,0.2,0.498549,0.997098


In [24]:
uxp.head(10)

,user_id,product_id,total_orders,times_last10,times_last10_ratio,reorder_ratio,Merged_Ratio
0,1,196,10,10.0,1.0,0.776480,7.764801
1,1,10258,9,9.0,0.9,0.713772,6.423947
2,1,10326,1,1.0,0.1,0.652009,0.652009
3,1,12427,10,10.0,1.0,0.740735,7.407350
4,1,13032,3,3.0,0.3,0.657158,1.971474
5,1,13176,2,2.0,0.2,0.832555,1.665110
6,1,14084,1,1.0,0.1,0.810982,0.810982
7,1,17122,1,1.0,0.1,0.675576,0.675576
8,1,25133,8,8.0,0.8,0.740155,5.921240
9,1,26088,2,2.0,0.2,0.539041,1.078082


In [62]:
uxp[['user_id','product_id','times_last10','reorder_ratio','Merged_Ratio']]

,user_id,product_id,times_last10,reorder_ratio,Merged_Ratio
0,1,196,10.0,0.776480,7.764801
1,1,10258,9.0,0.713772,6.423947
2,1,10326,1.0,0.652009,0.652009
3,1,12427,10.0,0.740735,7.407350
4,1,13032,3.0,0.657158,1.971474
...,...,...,...,...,...
13307948,206209,43961,3.0,0.630583,1.891748
13307949,206209,44325,1.0,0.401148,0.401148
13307950,206209,48370,1.0,0.699288,0.699288
13307951,206209,48697,1.0,0.357661,0.357661


In [25]:
recommed_df=uxp[['user_id','product_id','Merged_Ratio']]

In [63]:
recommed_df.head(10)

,user_id,product_id,Merged_Ratio
0,1,196,7.764801
1,1,10258,6.423947
2,1,10326,0.652009
3,1,12427,7.407350
4,1,13032,1.971474
5,1,13176,1.665110
6,1,14084,0.810982
7,1,17122,0.675576
8,1,25133,5.921240
9,1,26088,1.078082


In [27]:
recommed_df2= recommed_df.merge(products,on='product_id',how='left')

In [28]:
recommed_df2.sample(3)

,user_id,product_id,Merged_Ratio,product_name,aisle_id,department_id
2824984,43866,9124,1.305184,Broccoli & Cheddar Bake Meal Bowl,38,1
5998955,92624,10246,NaN,Organic Celery Hearts,83,4
6236820,96270,22255,0.643210,Organic Nonfat Greek Yogurt With Lemon,120,16


In [29]:
recommed_df2.shape

(13307953, 6)

In [30]:
rec_df=recommed_df2[['product_name','user_id','Merged_Ratio']]

In [31]:
print(rec_df.shape)
print(rec_df.head())

(13307953, 3)
            product_name  user_id  Merged_Ratio
0                   Soda        1      7.764801
1             Pistachios        1      6.423947
2    Organic Fuji Apples        1      0.652009
3    Original Beef Jerky        1      7.407350
4  Cinnamon Toast Crunch        1      1.971474


## Filtering users with minimum of 80 orders

In [149]:
x=rec_df.groupby('user_id').count()['Merged_Ratio']>80
users_min50orders = x[x].index

In [150]:
rec_df.shape

(13307953, 3)

In [151]:
filtered_users= rec_df[rec_df['user_id'].isin(users_min50orders)]

In [152]:
filtered_users.shape

(3482403, 3)

## Filtering products with minimum of 30 orders

In [153]:
filtered_users.groupby('product_name').count()['Merged_Ratio']>=30

product_name
#2 Coffee Filters                                                   True
#2 Cone White Coffee Filters                                       False
#2 Mechanical Pencils                                              False
#4 Natural Brown Coffee Filters                                     True
& Go! Hazelnut Spread + Pretzel Sticks                             False
                                                                   ...  
with Xylitol Unwrapped Original Flavor 50 Sticks Sugar Free Gum    False
with Xylitol Unwrapped Spearmint 50 Sticks Sugar Free Gum          False
with Xylitol Watermelon Twist 18 Sticks Sugar Free Gum             False
with a Splash of Mango Coconut Water                               False
with a Splash of Pineapple Coconut Water                           False
Name: Merged_Ratio, Length: 46798, dtype: bool

In [154]:
y= filtered_users.groupby('product_name').count()['Merged_Ratio']>=30
popular_products = y[y].index


In [155]:
popular_products.shape

(10816,)

In [185]:
popular_products.head()

AttributeError: 'Index' object has no attribute 'head'

In [156]:
filtered_users.shape

(3482403, 3)

In [157]:
final_df=filtered_users[filtered_users['product_name'].isin(popular_products)]

In [158]:
final_df.shape

(3092964, 3)

In [159]:
final_df.head()

,product_name,user_id,Merged_Ratio
18,Organic Turkey Burgers,2,0.448502
19,Wild Albacore Tuna No Salt Added,2,0.534449
20,Cherry Pomegranate Greek Yogurt,2,4.394576
21,"The \""World's Best\"" Veggie Burger",2,2.165254
22,Garlic Pepper Ramen,2,0.792079


In [43]:
final_df=final_df.reset_index(drop=True)

In [160]:
final_df.shape

(3092964, 3)

In [186]:
### Product List

productslist = final_df['product_name'].unique()

# Creating a new DataFrame with the unique values as a column
productslist = pd.DataFrame(productslist, columns=['product_name'])

                                        product_name
0                             Organic Turkey Burgers
1                   Wild Albacore Tuna No Salt Added
2                    Cherry Pomegranate Greek Yogurt
3                 The \"World's Best\" Veggie Burger
4                                Garlic Pepper Ramen
...                                              ...
10811      Traditional Chicken & Herb Dumplings Soup
10812          Slow Roasted Lightly Seasoned Chick'n
10813            Original Light Ranch Salad Dressing
10814                             Frozen Raspberries
10815  Organic Half Baby Spinach & Half Baby Arugula

[10816 rows x 1 columns]


In [197]:
productslist=productslist.astype('string')

In [200]:
productslist.head()

,product_name
0,Organic Turkey Burgers
1,Wild Albacore Tuna No Salt Added
2,Cherry Pomegranate Greek Yogurt
3,"The \""World's Best\"" Veggie Burger"
4,Garlic Pepper Ramen


## Pivot Table

In [161]:
pt = final_df.pivot_table(index='product_name',columns='user_id',values='Merged_Ratio')

In [162]:
pt

user_id,2,10,14,19,31,35,43,77,95,110,...,206118,206126,206146,206157,206160,206165,206174,206193,206199,206203
product_name,,,,,,,,,,,,,,,,,,,,,
#2 Coffee Filters,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
#4 Natural Brown Coffee Filters,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
0% Fat Blueberry Greek Yogurt,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
0% Fat Free Organic Milk,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
0% Fat Organic Greek Vanilla Yogurt,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
of Hanover 100 Calorie Pretzels Mini,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
smartwater® Electrolyte Enhanced Water,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
vitaminwater® XXX Acai Blueberry Pomegranate,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [163]:
pt.fillna(0,inplace=True)

In [178]:
pt

user_id,2,10,14,19,31,35,43,77,95,110,...,206118,206126,206146,206157,206160,206165,206174,206193,206199,206203
product_name,,,,,,,,,,,,,,,,,,,,,
#2 Coffee Filters,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
#4 Natural Brown Coffee Filters,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
0% Fat Blueberry Greek Yogurt,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
0% Fat Free Organic Milk,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
0% Fat Organic Greek Vanilla Yogurt,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
of Hanover 100 Calorie Pretzels Mini,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
smartwater® Electrolyte Enhanced Water,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
vitaminwater® XXX Acai Blueberry Pomegranate,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [170]:
pt.sample(5)

user_id,2,10,14,19,31,35,43,77,95,110,...,206118,206126,206146,206157,206160,206165,206174,206193,206199,206203
product_name,,,,,,,,,,,,,,,,,,,,,
Pretzel In Milk Chocolate Bar,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
American Blend Salad,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
Froot Loops Family Size Cereal,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
Organic Beans Garbanzo,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
Essential Greens With Lime Vegetable and Fruit Juice Blend,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [50]:
from sklearn.metrics.pairwise import cosine_similarity

In [164]:
similarity_scores=cosine_similarity(pt)

In [165]:
def recommendation(product_name):
    # index fetch
    index = np.where(pt.index==product_name)[0][0]
    similar_items = sorted(list(enumerate(similarity_scores[index])),key=lambda x:x[1],reverse=True)[1:11]
    
    # data = []
    for i in similar_items:
        print(pt.index[i[0]])
        # item = []
        # temp_df = books[books['Book-Title'] == pt.index[i[0]]]
        # item.extend(list(temp_df.drop_duplicates('Book-Title')['Book-Title'].values))
        # item.extend(list(temp_df.drop_duplicates('Book-Title')['Book-Author'].values))
        # item.extend(list(temp_df.drop_duplicates('Book-Title')['Image-URL-M'].values))
        
        # data.append(item)
    
    # return 

In [169]:
recommendation('French Green Beans')

Organic Tree Ripened Plums
Organic Black Beans Ready To Eat In Pouch
90% Lean Ground Beef
Limes
70% Dark Chocolate Bar
Prepared Horseradish
Burrito Size Flour Tortillas
Large Lemon
Fresh Cauliflower
Banana


In [171]:
recommendation('Froot Loops Family Size Cereal')


Thick Cut Bacon
Deluxe Blueberry Bagels
Whipping Cream Ultra Pasteurized
Spirals Macaroni & Cheese Dinner
Chewy Candies, Flavored, Fierce Cinnamon
Butter Top White Bread
Eggo Blueberry Waffles
Steamfresh Premium Selects Frozen Gold & White Corn
Blueberry Muffins
Breaded Mozzarella Sticks


In [172]:
recommendation('Zucchini Squash')

Red Pepper
Ultra Half & Half
Celery
Cilantro Bunch
Chocolate Cheerios Cereal
Potatoes Sweet
Goat Cheese
Cut Corn
Everyday Original Turkey Bacon
Beef Bouillon Cubes


In [221]:
(pt.index=='Organic Turkey Burgers')

array([False, False, False, ..., False, False, False])

In [194]:
pt.index.get_loc(product_name)

7131

# Due To Large Data Files- Cutting down on size to build Website

## Filtering users with minimum of 60 orders

In [ ]:
x2=rec_df.groupby('user_id').count()['Merged_Ratio']>60
users_min100orders = x2[x2].index

In [ ]:
filtered_users2= rec_df[rec_df['user_id'].isin(users_min100orders)]

In [ ]:
filtered_users2.shape

## Filtering products with minimum of 30 orders

In [ ]:
y2= filtered_users2.groupby('product_name').count()['Merged_Ratio']>=30
popular_products2 = y2[y2].index

In [ ]:
popular_products2.shape

In [ ]:
popular_products2

In [ ]:
final_df2=filtered_users2[filtered_users2['product_name'].isin(popular_products2)]

In [ ]:
final_df2.shape

In [ ]:
final_df2=final_df2.reset_index(drop=True)

## Pivot Table

In [ ]:
pt2 = final_df2.pivot_table(index='product_name',columns='user_id',values='Merged_Ratio')

In [ ]:
pt2.fillna(0,inplace=True)

In [ ]:
pt2.sample(5)

In [ ]:
similarity_scores2=cosine_similarity(pt2)

In [ ]:
def recommendation2(product_name):
    # index fetch
    index = np.where(pt2.index==product_name)[0][0]
    similar_items = sorted(list(enumerate(similarity_scores2[index])),key=lambda x:x[1],reverse=True)[1:11]
    
    # data = []
    for i in similar_items:
        print(pt2.index[i[0]])
        # item = []
        # temp_df = books[books['Book-Title'] == pt.index[i[0]]]
        # item.extend(list(temp_df.drop_duplicates('Book-Title')['Book-Title'].values))
        # item.extend(list(temp_df.drop_duplicates('Book-Title')['Book-Author'].values))
        # item.extend(list(temp_df.drop_duplicates('Book-Title')['Image-URL-M'].values))
        
        # data.append(item)
    
    # return 

In [ ]:
recommendation2('Zucchini Squash')

# Pickle

In [173]:
import pickle

In [201]:
pickle.dump(productslist.to_dict(),open('rec_productlist.pkl','wb'))

In [202]:
productslist.iloc[2].product_name

'Cherry Pomegranate Greek Yogurt'

In [174]:
pt=pt.astype('float32')

In [183]:
pickle.dump(pt2,open('pt.pkl','wb'))

In [176]:
similarity_scores=similarity_scores.astype('float32')

In [177]:
pickle.dump(similarity_scores,open('rec_similarity.pkl','wb'))

In [182]:
pt2=pt.head(1000)

In [206]:
products_dicts = pickle.load(open('products_with_recommendations.pkl', 'rb'))

In [245]:
products_dicts.sample(10)

,product_name
98,Almond Flour Fine Ground Sea Salt Crackers
697,All Natural Whole Wheat Bread
323,Beefsteak Tomato
811,All Natural Plain Yogurt
47,100% Organic Diced Tomatoes
934,BBQ Chopped Salad
833,3D White Mint Toothpaste
784,Bagel Chips
689,Beef Pot Pie
754,Blueberry & Purple Carrot Teething Wafers


In [223]:
products_dicts.shape

(1000, 1)